# Calculating Spike Rates From Lay Files

MNE has some useful features that will allow us to quickly read the spike data and calculate statistics of interest.

**Note:** mne must have version 0.23 or else there is a bug in the annotation reader

## Import Tools

**Alana T**

**NCSL**

**Last updated: 8/14/2021**

In [1]:
%reset -f
from pathlib import Path
import collections
from collections import defaultdict
from collections import Counter
import glob
import mne
%matplotlib inline
import matplotlib as pml
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas.core.common import flatten
import os
import random
import re
import seaborn as sns
from sklearn.model_selection import(
    train_test_split,
    StratifiedShuffleSplit
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    auc,
    roc_curve,
    accuracy_score
)
from sklearn.feature_selection import SelectKBest, chi2
import statsmodels.api as sm

# Prepare xlsx file for dataframe usage

In [ ]:
# Read excel file from path 'xlsdat' into a dataframe, dropping extraneous columns
# Change file path to your data folder
xlsdat = r"C:\Users\atill\OneDrive - Johns Hopkins\SarmaLab\spike_detection\JHU_scalp_clinical_datasheet_raw.xlsx"
df = pd.read_excel(xlsdat)
# Drop extraneous columns
df.drop(df.columns[3:len(df.columns)], axis=1, inplace=True) 
df = df.loc[df.index.repeat(df.number_datasets)]
df = df.reset_index()
df = df.drop('number_datasets',1)

## Relabel patient_id data to correspond to patient condition (0: normal, 0.5: norm-ep, 1: abnorm-ep)

In [ ]:
filter = lambda x : 0 if x < 100 else (0.5 if x < 200 else 1)
df['patient_id'] = df.patient_id.apply(filter)

## Define a function for analyzing .lay files

### Run analyze_lay to find spike data by patient and spike data by patient by patient channel

### input is a file path where lay files are located

### output is two dictionaries containing average spike rate for a pt and the spike rate on individual channels

In [ ]:
# Spike analysis code from Patrick: 
# Extract important information from the object (annotations, duration of recording, number of total channels)
def analyze_lay(lay_fpath):
    raw = mne.io.read_raw_persyst(lay_fpath)
    verbose = True
    annotations = raw.annotations
    mins = raw.n_times/(60*raw.info['sfreq'])  # We will be calculating spikes/min in each channel
    n_chs = raw.info['nchan']
    # Retrieve spike annotations, get rid of information except channel name
    spike_channels = [a['description'].split(' ')[1] for a in annotations if 'spike' in a['description']]
    # Extract relevant rates
    def count_spikes(spike_list):
        spike_dict = {}
        for ch in list(set(spike_list)):
            spike_dict.update({ch: spike_list.count(ch)})
        return spike_dict
    spike_counts = count_spikes(spike_channels)
    spike_rates = {}
    for key, val in spike_counts.items():
        spike_rates[key] = val / mins
    total_spike_rate = sum(spike_counts.values()) / (n_chs*mins)  # normalizing by total time and number of channels
    return [total_spike_rate, spike_rates]

## Define a function to extract important information from lay data and appent it to a dataframe

### Inputs are a dictionary for label: data (a_dict, e.g. hospital_id: avg. spikes/min), a dataframe to append data to (a_df, e.g. df, with a dictionary to df mapping of 'hospital_id' (can be changed)), and a section name for that dataframe (a_secname, e.g. 'spikes')

### Output is a dataframe with a new column in a_df mapped from a_dict under the column name a_secname

In [ ]:
def numbidx(a_dict, a_df, a_secname):
    dup = []
    a_dict2 = a_dict.copy()
    for key, value in a_dict.items():
        for k in a_dict.keys():
            if ((key[0:2] == k[0:2]) and (key != k)):
                if key[0:2] not in dup:
                    dup.append(key[0:2])
                a_dict2[key] = [a_dict[key], a_dict[k]]
        if '-' in key[0:2]:
            a_dict2[int(key[0:1])] = a_dict2[key]
            a_dict2.pop(key)
        else:
            a_dict2[int(key[0:2])] = a_dict2[key]
            a_dict2.pop(key)
    a_df[a_secname] = a_df['hospital_id'].map(a_dict2)
    a_df2 = a_df.copy()
    for k in dup:
        for index in range(0,len(a_df)-1):
            if (a_df.iloc[index].hospital_id == int(k)) and (a_df.iloc[index+1].hospital_id == int(k)):
                val = a_df.iloc[index][a_secname]
                a_df2.at[index, a_secname] = val[0]
                a_df2.at[index+1, a_secname] = val[1]
    return a_df2

## Define a function to report TP, FP, TN, FN (performance measure)

### Inputs are actual labels (y_actual0), predicted labels (y_hat0), and a theta threshold (theta)

### Modifies each point from the input into 1 or 0 after comparison to a theta via .apply (e.g. 0.6 --> 1, 0.2 -->0)

### Outputs are TP, FP, TN, FN counts

In [ ]:
def perf_measure(y_actual0, y_hat0, theta):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    binry = list()
    y_hat = y_hat0.to_list()
    y_actual = y_actual0.to_list()
    
    for i in range(len(y_hat)):
        if y_hat[i] <= theta:
            binry.append(0)
        elif y_hat[i] >= theta:
            binry.append(1)
    
    for i in range(len(binry)): 
        if y_actual[i]==binry[i]==1:
           TP += 1
        if binry[i]==1 and y_actual[i]!=binry[i]:
           FP += 1
        if y_actual[i]==binry[i]==0:
           TN += 1
        if binry[i]==0 and y_actual[i]!=binry[i]:
           FN += 1

    return(TP, FP, TN, FN)

## Load in lay data and analyze it with analyze_lay function

In [ ]:
# Change this path to where lay files are located
path = r"C:\Users\atill\OneDrive - Johns Hopkins\SarmaLab\spike_detection\data"
lay_files = glob.glob(path + "/**/*.lay", recursive = True)
filename = [os.path.split(lay_files[i])[-1][:-4] for i in range(len(lay_files))]
# Run analyze_lay to find spike data by patient and spike data by patient by channel 
spikedat = [analyze_lay(lay_files[j])[0] for j in range(len(lay_files))]
spikechrates = [analyze_lay(lay_files[j])[1] for j in range(len(lay_files))]
spikedict = dict(zip(filename, spikedat))
# Append 'spikes' column (spikes / min / pt) to df
df3 = numbidx(spikedict, df, 'spikes')

# Feature 1: Max spikes (maxspikech)

In [3]:
spikechdict = dict(zip(filename, spikechrates))
spikechdict2 = spikechdict.copy()
spikechdict2.keys()
# Find max spike rate value in spikechdict and append to spikechdict2 (a dictionary containing maximum spike rate) 
for key, value in spikechdict.items():
    if spikechdict2[key] == {}:
        spikechdict2[key] = {'0':0}
    spikechdict2[key] = max(spikechdict2[key].values())

NameError: name 'filename' is not defined

# Extract channels by electrode lobe location

In [ ]:
# Separate spike data in pt groups, append spikechdict data by temporal, frontal, occipital, and parietal lobes 
# into pt grouped dicts 
chdictnorm, chdictepnorm, chdictepabnorm = defaultdict(list), defaultdict(list), defaultdict(list)
indnorm = df3[df3['patient_id']==0.0].hospital_id.values.tolist()
indep0 = df3[df3['patient_id']==1.0].hospital_id.values.tolist()
# Enclude these patients in the next line due to being outliers, having 0 spikes, etc criteria
filter = [3,48,57,63] 
filtered_list = []
indnormidx, indabnormidx, grps = [], [], []
indep = [element for element in indep0 if element not in filter]
indnormidx = [str(i) + '-clip' for i in indnorm]
indabnormidx = [str(i) + '-clip' for i in indep0]
grps = indnorm + indep0
indnormep0 = [i for i in range(1,87) if i not in grps]
indnormepidx = [str(i) + '-clip' for i in range(1,87) if i not in grps]
for i in range(len(indnormidx)-1):
    if indnorm[i] > 76:
        indnormidx[i] = str(indnorm[i]) + 'awake-clip'
        if indnorm[i] in [76, 79, 81, 83, 84]:
            indnormidx.append(str(indnorm[i]) + 'sleep-clip')
        elif indnorm[i] == 77:
            indnormidx.append(str(indnorm[i]) + 'lightsleep-clip')
for i in range(len(indabnormidx)-1):
    if indep0[i] > 76:
        indabnormidx[i] = str(indep0[i]) + 'awake-clip'
        if indep0[i] in [76, 79, 81, 83, 84]:
            indabnormidx.append(str(indep0[i]) + 'sleep-clip')
for i in range(len(indnormepidx)-1):
    if indnormep0[i] > 76:
        indnormepidx[i] = str(indnormep0[i]) + 'awake-clip'
        if indnormep0[i] in [76, 79, 81, 83, 84, 86]:
            indnormepidx.append(str(indnormep0[i]) + 'sleep-clip')
# Each pt group contains spike data grouped by first two characters in channel name if channel name begins 
# with t (temporal), f (frontal), o (occipital), or p (parietal)
for i in spikechdict.keys():
    for k in spikechdict[i].keys():
        if k[0] in ['t', 'f', 'o', 'p']:
            if i in indnormidx:
                chdictnorm[k[0:2]].append(spikechdict[i][k])
            elif i in indnormepidx:
                chdictepnorm[k[0:2]].append(spikechdict[i][k])
            elif i in indabnormidx:
                chdictepabnorm[k[0:2]].append(spikechdict[i][k])

# Consolidate lobe data (t, f, o, p)

### tL, tR, meansrest (of brain) and readied to be appended to df as features in this block

In [ ]:
sums = dict(Counter(t3) + Counter(t5))
meanstleft = {k: sums[k] / float((k in t3) + (k in t5)) for k in sums}
sums2 = dict(Counter(t4) + Counter(t6))
meanstrght = {k: sums2[k] / float((k in t4) + (k in t6)) for k in sums2}
tR = t3.copy()
tL = t4.copy()
for key in meanstrght:
    tR[key] = meanstrght[key]
for key in meanstleft:
    tL[key] = meanstleft[key]
sums3 = dict(Counter(tL) + Counter(tR))
meansall = {k: sums3[k] / float((k in tL) + (k in tR)) for k in sums3}
tall = tR.copy()
for key in meansall:
    tall[key] = meansall[key]

sums4 = dict(Counter(f) + Counter(o) + Counter(p))
meansrest = {k: sums4[k] / float((k in f) + (k in o) + (k in p)) for k in sums4}
meansrest2 = p.copy()
for key in meansrest:
    meansrest2[key] = meansrest[key]

# Plot average spikes / min per channel per patient group

## Adam put very well during the presentation that there are many more channels with the first starting character, and that the jutter that I added can make the plots harder to understand (e.g. overall t should see analogous spikes from t3, t4..etc.)

In [ ]:
for i in spikechdict.keys():
    for k in spikechdict[i].keys():
        if k[0] in ['t', 'f', 'o', 'p']:
            if i in indnormidx:
                chdictnorm[k[0:1]].append(spikechdict[i][k])
            elif i in indnormepidx:
                chdictepnorm[k[0:1]].append(spikechdict[i][k])
            elif i in indabnormidx:
                chdictepabnorm[k[0:1]].append(spikechdict[i][k])
offst = 0.0 # This line can provide a slight random jitter for data visualization (0.015 was used for the presentation, but here it is 0)
xn, yn = zip(*((float(x)+offst*random.randint(1,5), k) for k in sorted(chdictnorm) for x in chdictnorm[k]))
xen, yen = zip(*((float(x)+offst*5*random.randint(1,5), k) for k in sorted(chdictepnorm) for x in chdictepnorm[k]))
xabe, yabe = zip(*((float(x)+offst*5*random.randint(1,5), k) for k in sorted(chdictepabnorm) for x in chdictepabnorm[k]))
fig, axs = plt.subplots(1, 3)
axs[0].scatter(yn, xn, c="k", alpha=1/2, s=90)
axs[0].set_title('norm')
plt.grid(linestyle='dotted')
axs[1].scatter(yen, xen, c="c", alpha=1/2, s=90)
axs[1].set_title('ep-norm')
axs[0].grid(linestyle='dotted')
axs[1].grid(linestyle='dotted')
axs[2].set_title('ep-abnorm')
axs[2].scatter(yabe, xabe, c="m", alpha=1/2, s=90)
plt.suptitle('Average spikes / min per channel per grp')
fig.text(0.5, 0.04, 'lobe', ha='center')
fig.text(0.04, 0.5, 'avg spikes / min', va='center', rotation='vertical')
plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams['figure.dpi'] = 100
plt.grid(linestyle='dotted')
plt.show()

## Assign feature candidates (lobe data and others) to dataframe

In [ ]:
df3 = numbidx(spikechdict2, df3, 'maxspikech')
df3 = numbidx(tall, df3, 't_lobespk')
df3 = numbidx(tR, df3, 't_Rlobespk')
df3 = numbidx(tL, df3, 't_Llobespk')
df3 = numbidx(meansrest2, df3, 'f_o_p_lobespk')
df3 = df3.fillna(0.0)
#df3 = numbidx(o, df3, 'o_lobe') # meansrest can be split up in this manner (e.g. o can be separated in this way)

## Visualization of section of df

In [ ]:
df3.tail(20)

## Divide pt groups in dataframe

In [ ]:
# Select equal sets of normal and abnormal (epileptic)
rindnorm=[]
indnorm = df3[df3['patient_id']==0.0].index.values.tolist()
indnormep = df3[df3['patient_id']==0.5].index.values.tolist()
indep0 = df3[df3['patient_id']==1.0].index.values.tolist()
filter = [3,48,57,63]
filtered_list = []
indep = [element for element in indep0 if element not in filter]
rindnorm = random.sample(indnorm, round(len(indep)/2))
# Extend list of normal-nonepilepsy with normal-epilepsy
# This was updated from the presentation on 8/9. 
rindnorm.extend(random.sample(indnormep, round(len(indep)/2)))

# Select features

In [ ]:
# df4 contains labels for normal-nonepilepsy=0, normal-epilepsy=0.5, abnormal-epilepsy=1.0
df4 = df3.copy()
# df3 further binarizes label for logistic regression as normal-nonepilepsy,normal-epilepsy=0, abnormal-epilepsy=1.0 
# via filter
filter = lambda x : 0 if x < 1 else 1
df3['patient_id'] = df3.patient_id.apply(filter)

In [ ]:
# Select top 4 features via SelectKBest and chi2 method 
selector = SelectKBest(chi2, k = 4)
x = df3[df3.columns[3:]].iloc[rindnorm + indep]
y = df3[['patient_id']].iloc[rindnorm + indep]
X_new = selector.fit_transform(x, y)
names = x.columns.values[selector.get_support()]
scores = selector.scores_[selector.get_support()]
pvalues = selector.pvalues_[selector.get_support()]
names_scores = list(zip(names, pvalues))

## Describe chosen features

In [ ]:
print(names)
print(scores)
print(names_scores)
print(pvalues)

# Run logistic regression

In [ ]:
# Establish x (features) and y (label) vectors
x = df3[names].iloc[rindnorm + indep]
y = df3[['patient_id']].iloc[rindnorm + indep] 

In [ ]:
# Apply stratifiedshuffle split on x, y data for train, test groups
xysplit = StratifiedShuffleSplit(n_splits=10, test_size=0.2) # change n_splits for n groups to test
xysplit.get_n_splits(x,y)
x_train, x_test, y_train, y_test = [], [], [], []
for train_index, test_index in xysplit.split(x, y):
    x_train.append(x[names].iloc[train_index])
    x_test.append(x[names].iloc[test_index])
    y_train.append(y[['patient_id']].iloc[train_index])
    y_test.append(y[['patient_id']].iloc[test_index])

## Keep an example test, train grouping using below block

### model perfomance varies greatly depending on the luck of the train/test group selected at this point

In [ ]:
x_train2 = x_train
x_test2 = x_test
y_train2 = y_train
y_test2 = y_test

# Run logistic regression

## Apply sm.Logit with a constant and no penalty term

In [ ]:
y_pred = [0]*len(y_test)
for i in range(len(y_test)):
    # Modify features to use via manipulating x_train[i], (e.g. max_spikes only is x_train[i].max_spikes)
    model = sm.Logit(y_train[i], sm.add_constant(x_train[i].astype(float)),penalty=None).fit()
    print(model.summary()) # Can comment this line if model summary not desired
    y_pred[i] = model.predict(exog=sm.add_constant(x_test[i].astype(float)))
    print(i, model.params) # Can comment this and the next line if model parameter display not desired
    print(i, model.pvalues)

## Describe feature characteristics as a boxplot

In [ ]:
box = df4.boxplot(column=names.tolist(), by='patient_id', grid=False)
plt.rcParams['figure.figsize'] = [10, 10]

In [ ]:
box = df3.boxplot(column=names.tolist(), by='patient_id', grid=False)
plt.rcParams['figure.figsize'] = [10, 10]

## Add predicted values to df

In [ ]:
dfp = pd.DataFrame(df3.patient_id, df3.hospital_id)
dfp = dfp.reset_index()
dfp['p'] = np.NaN
for i in range(len(y_pred)):    
    dfp['p'].update(dfp['hospital_id'].map(y_pred[i].to_dict()))
dfp = dfp.dropna()

## Visualize feature by patient group (feel free to change y= to another value)

### Colorcoding by epilepsy group (0.0, 0.5) not incorporated

In [ ]:
ax = sns.boxplot(x="patient_id", y="f_o_p_lobespk", data=df4, showfliers = False, color='white', width=0.25)
ax = sns.swarmplot(x="patient_id", y="f_o_p_lobespk", data=df4, color=".25")
plt.show()

## Show ROC curves

### Confidence interval visualization for ROCs not incorporated yet

In [ ]:
aucs, fprs, tprs, threshs = list(), list(), list(), list()
ax = plt.axes()
for i in range(len(y_test)):
    fpr, tpr, thresh = roc_curve(y_test[i], y_pred[i])
    fprs.append(fpr), tprs.append(tpr), threshs.append(thresh)
    aucx = roc_auc_score(y_test[i], y_pred[i])
    aucs.append(aucx)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label='LogReg')
    plt.xlabel('False Pos Rate')
    plt.ylabel('True Pos Rate')
    plt.title('LogReg ROC Curve for Normal/Abnormal-Epilepsy Groups')
    plt.rcParams['figure.figsize'] = [6, 5]
    plt.show

## Find optimal threshold via argmax of tprs - fprs and max accuracy

In [ ]:
Acclist, optimal_threshs = list(), list()
for i in range(len(fprs)):
    optimal_idx = np.argmax(tprs[i] - fprs[i])
    optimal_threshs.append(threshs[i][optimal_idx])
    TP, FP, TN, FN = perf_measure(dfp.patient_id, dfp.p, optimal_threshs[i])
    Acclist.append((TP + TN)/ (TP + FP + TN + FN))
idx = Acclist.index(max(Acclist))
optimal_threshold = optimal_threshs[idx]

## Report accuracy, sensitivity, and specificity of model on tested data

### Accuracy has diminished since the addition of normal-epilepsy groups, unfortunately --

### Incorporation of spectral data, artifact removal, and/or feature revision may address this

#### Modification of 'rindnorm' group to exclude normal-epilepsy patients returns what was seen in the presentation, with luck of the train/test group

In [ ]:
TP, FP, TN, FN = perf_measure(dfp.patient_id, dfp.p, optimal_threshold)
Acc = (TP + TN)/ (TP + FP + TN + FN)
Sens = TP / (TP + FN)
Spec = TN / (TN + FP)
print('The accuracy of tested data is ', Acc, '. The sensitivity of tested data is ', Sens, '. The specificity of tested data is ', Spec, '.', 'Mean AUC is', np.mean(aucs), '.')

## Visualize prob(epilepsy) by patient group
### Colorcoding by epilepsy group (0.0 and 0.5) not incorporated

In [ ]:
dfp.boxplot(column='p', by='patient_id', grid=False)
ax = sns.swarmplot(x="patient_id", y="p", data=dfp, color=".25") 
plt.ylabel('P(epilepsy)');
plt.plot([-1, 2], [optimal_threshold, optimal_threshold], 'm--')
print('Optimal theta calculated as', optimal_threshold)